In [1]:
# pip install pycountry
import pycountry
# pycountry.countries
# pycountry.languages
import re
from pathlib import Path
import pandas as pd

data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs/intermediate")
file_name = data_folder / "B_Phase1.pkl"
df = pd.read_pickle(file_name)

In [2]:
#Education - extract the most recent education school name

# further seperate
ETCs = [re.split(r"\n", content) if i < 10 \
        else re.split(r"\, \d{4}", content) if i == 31 \
        else re.split(r"\n\? ", content) for i, content in enumerate(df['ETC'])]
ETCs_1 = []

for i in ETCs:
    temp = list(filter(("".__ne__), i))
    temp = list(filter((" ".__ne__), temp))
    temp = list(filter((' Training, and Certifications'.__ne__), temp))
    for j in range(len(temp)):
        temp[j] = temp[j].strip()
    ETCs_1.append(temp)
    
# Special Case Debugging
ETCs_1[3] = re.split(r" (?=B.A.)", ETCs_1[3][0])
ETCs_1[24] = [re.split(r"\n", ETCs_1[24][0])[1]]
ETCs_1[25] = re.split(r"(?<=\, \d{4}) |\n", ETCs_1[25][0])[1:]
ETCs_1[26][1] = re.sub(r"\t", " ", ETCs_1[26][1])
ETCs_1[29] = re.split(r"(?<=\, \d{4})\s", ETCs_1[29][0])

ETCs_1[30] = re.split(r"(?<=\, \d{4})\s", ETCs_1[30][0])
ETCs_1[30][2:3] = re.split(r"(?<=\, \d{4})\; ", ETCs_1[30][2])
ETCs_1[30][2] = re.sub(r"\nAdditional Relevant Training: ", "", ETCs_1[30][2])

In [3]:
#Extract Certifications/Licenses

certificates = {"PMP: Project Management Professional": ["Project Management Professional"],
                "CPC: Certified Professional Coach": ["Professional Certified Coach", " Certified Professional Coach"],
                "Residency": ["Residency"],
                "CPA: Certified Public Accountant": ["Certified Public Accountant"],
                "ACCA: Association of Chartered Certified Accountants": ["ACCA"] 
               }

def CheckCertificate (etc):
    p_certificate = []
    for i in certificates:
        for title in certificates[i]:
            if title in etc:
                p_certificate.append(i)
                break
    return p_certificate

Certificates = [CheckCertificate(p) for p in df["ETC"]]

In [4]:
#Extract the highest degree
degrees = {"Ph.D.": ["Postdoctoral", "PhD", "Ph.D.", "Doctor of Philosophy", "Doctoral", "Doctor of Education", "Ed.D.",\
                     "D.D.S.", "DDS", "Doctor of Literature", "Doctor in Public Health"],
           "M.D.": ["M.D.", "Doctor of Medicine", "Doctorate in Medicine"],
           "MBA": ["MBA"],
           "Master": ["Master", "MPH", "M.P.H.", "M.S.", "M.S.N.", "M.A.", "MA", "M.P.", "Graduate Studies"],
           "Bachelor": ["Bachelor", "BA", "B.A.","A.B.", "B.S.", "BS", "Undergraduate"]}

def CheckDegree (p_edu):
    p_degree = []
    for i in degrees:
        for title in degrees[i]:
            if title in p_edu:
                p_degree.append(i)
                break
    return p_degree

Degrees = [CheckDegree(p) for p in df["ETC"]]

In [5]:

# seperate ETC into education and training/certificate
# edu = []
# tr_cer = []
# pattern_edu = r"University|Master|School|College|DrPH|Bachelor"

# for i in ETCs_1: # for every consultant
#     edu_temp = []
#     tr_cer_temp = []
#     for j in i:
# #         print(j, re.match(pattern_edu, j))
#         if re.search(pattern_edu, j):
#             edu_temp.append(j)
#         else:
#             tr_cer_temp.append(j)
#     edu.append(edu_temp)
#     tr_cer.append(tr_cer_temp)


In [6]:
#Language

ppls_noname = list(df['ppls_data'])

# now an explicit list of languages is used for speed, but packages pycountry can be used in the future
lang_dic = ['Amharic', 'Arabic', 'Bemba', 'Chinese', 'Creole', 'Danish', 'French', 'German', 'Hindi',\
            'Kinyarwanda', 'Kirundi', 'Kiswahili', 'Luo', 'Malayalam', 'Nyanja', 'Portuguese', 'Russian',\
            'Setswana', 'Spanish', 'Swahili', 'Tagalog', 'Tigrigna', 'Tigrinya', 'Yoruba']

lang = []

for i in ppls_noname:
    temp = [j for j in lang_dic if re.search(j, i)]
    lang.append(temp)

In [7]:
#Countries worked/visited/studied
cnty = []
for i in ppls_noname:
    temp = []
    for country in pycountry.countries:
        if re.search(country.name, i):
            temp.append(country.name)
    cnty.append(temp)

In [8]:
#Underserved population worked with

undsrvd_dic = ['HIV', 'homeless', 'low income', 'poverty', 'poor', 'LGBT', 'transgender', 'trafficking',\
           'infants', 'children', 'youth', 'elder', 'disabilit', 'rural', 'low literacy']

#8/12/19/40/41/49/56 disabilities

undsrvd_pop = []
for i in ppls_noname:
    temp = [j for j in undsrvd_dic if re.search(j, i, re.IGNORECASE)]
    temp = ['disabilities' if t == "disabilit" else t for t in temp]
    temp = ['low income' if t == 'poor' else t for t in temp]
    temp = ['low income' if t == 'poverty' else t for t in temp]
    temp = list(set(temp))
    undsrvd_pop.append(temp)
# undsrvd_pop

In [9]:
#Remove ETC, add Education and training/certificate columns
# df["Education"] = edu
# df["Training/Certificate"] = tr_cer
# del df["ETC"]

#Add languages, countries worked, underserved population
df['ETC'] = ETCs_1
df['Important Certificates'] = Certificates
df["Degrees"] = Degrees
df["Languages"] = lang
df["Countries worked/studied"] = cnty
df["Underserved population worked with"] = undsrvd_pop

In [10]:
#Title Cleaning

for i in [7, 8, 9, 20, 21, 25, 29]:
    df["Title"][i] = re.sub(r"\s\/\s", r"/", str(df["Title"][i]))

df["Title"][27] = re.sub(r"\s\([\w\s]+\)", "", str(df["Title"][27]))

In [11]:
#QE Cleaning

for i, w in enumerate(df["Qualification and Experience"]):
    df["Qualification and Experience"][i] = w.strip()

In [12]:
# Career Summary Cleaning

for i, w in enumerate(df['Career Summary']):
    temp = re.sub(r"\n", " ", w)
    df['Career Summary'][i] = re.sub(r"(Career )?Summary", "", temp).strip()


In [13]:
# Storing

file_name = data_folder / "B_Phase2.pkl"
df.to_pickle(file_name)